In [1]:
import utils

utils.reload()

In [2]:
from vllm import LLM, SamplingParams
import os

In [3]:
model_name_or_path = "/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9"
tokenizer_name_or_path = "/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [4]:
llm = LLM(
    model=model_name_or_path,
    tokenizer=tokenizer_name_or_path,
    tokenizer_mode="auto",
    trust_remote_code=True,
    tensor_parallel_size=1,
    dtype="auto",
    seed=0,
)

INFO 08-25 15:48:16 llm_engine.py:70] Initializing an LLM engine with config: model='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9', tokenizer='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470', tokenizer_mode=auto, trust_remote_code=True, dtype=torch.float16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, seed=0)
INFO 08-25 15:48:16 tokenizer.py:29] For some LLaMA-based models, initializing the fast tokenizer may take a long time. To eliminate the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.
INFO 08-25 15:48:21 llm_engine.py:196] # GPU blocks: 7439, # CPU blocks: 512


In [6]:
# prompts = [
#     "Hello, my name is",
#     "The president of the United States is",
#     "The capital of France is",
#     "The future of AI is",
# ]
problem_solution_hierarchical_samples = utils.load_pickle(
    utils.gpt4_generated_problem_solution_hierarchical_samples_path_wo_basename + ".pkl"
)

In [11]:
def problem_solutions2solution_texts(problem_solution):
    for problem_sample in problem_solution:
        problem_text = problem_sample["problem"]
        for solution_sample in problem_sample["solutions"]:
            solution_text = problem_text
            for step in solution_sample["steps"]:
                solution_text += "\n" + step
            yield solution_text


solution_texts = list(
    problem_solutions2solution_texts(problem_solution_hierarchical_samples)
)

print(len(solution_texts))

815631
988


In [13]:
sampling_params = SamplingParams(**utils.generation_config)

# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(solution_texts, sampling_params)
# Print the outputs.
# for output in outputs:
#     prompt = output.prompt
#     generated_text = output.outputs[0].text
#     print(
#         f"""
# Prompt: {prompt!r}
# Generated text: {generated_text!r}
# """
#     )

KeyboardInterrupt: 